---
syncID: 
title: "Querying Taxonomy Data with NEON API and Python"
description: "Querying the 'taxonomy/' NEON API endpoint with Python and navigating the response"
dateCreated: 2020-04-24
authors: Maxwell J. Burner
contributors: 
estimatedTime:
packagesLibraries: requests, json
topics: api
languagesTool: python
dataProduct:
code1: 
tutorialSeries: python-neon-api-series
urlTitle: neon_api_taxonomy
---

In this tutorial we will learn to query the *taxonomy/* endpoint of the NEON API using Python.

<div id="ds-objectives" markdown="1">

### Objectives
After completing this tutorial, you will be able to:

* Query the taxonomy endpoint of the NEON API to obtain taxonomic data
* Search NEON taxonomic data using different criteria
* Use the various options of the taxonomy endpoint to customized the results of a call
* Navigate the data returned by a call to the taxonomy endpoint of the NEON API
* Navigate the parent-child relationships between NEON locations


### Install Python Packages

* **requests**
* **json** 



</div>

In this tutorial we will learn to use Python and the *taxonomy/* endpoint of the NEON API to query information from NEON's taxonomic data. NEON maintains a great deal of taxonomic data, used in species identification during field research. NEON taxonomy data can be obtained through the API, or through an interactive interface called the [Taxon Viewer](http://data.neonscience.org/static/taxon.html).

NEON taxonomy data includes 


## Making the Request

Unlike other endpoints, the *locations/* endpoint does not take a single target in its URL. Instead, the query can make use of a number of different options, which are named and assigend values in the url. Each option is assigned a value with an equals, for example 'family=Pineceae'; these are placed after a '?' at the end of the URL, linked to each other by '&' signs.

Each call must have one of the following options, but cannot use multiple:
* **taxonTypeCode**, a four-letter that indicates which NEON taxonomy is being queried, such as FISH or BIRD
* One of the major taxonomic ranks from genus through kingdom
* **scientificName** a specific name of format genus + specific epithet + (authority); this is used to search for one exact results

In addition, any number of the following options can also be added to modify the results of the query:
* **verbose** takes a 'true' for a more detailed response or 'false' for a shorter response
* **offset** takes an integer indicating the number of starting rows of the list of results to skip; the default is 50
* **limit** takes an integer indicating the maximum length of the list returned; the default is 50

Let's request data on up to 20 members of the Pine family, skipping none, with the short response.

In [12]:
import requests
import json

In [13]:
#Choose values for each option
SERVER = 'http://data.neonscience.org/api/v0/'
FAMILY = 'Pinaceae'
OFFSET = 0
LIMIT = 20
VERBOSE = 'false'

In [14]:
#Create 'options' portion of API call
OPTIONS = '?family={family}&offset={offset}&limit={limit}&verbose={verbose}'.format(
    family = FAMILY,
    offset = OFFSET,
    limit = LIMIT,
    verbose = VERBOSE)
print(OPTIONS)

?family=Pinaceae&offset=0&limit=20&verbose=false


In [15]:
#Make request
pine_req = requests.get(SERVER+'taxonomy/'+OPTIONS)
pine_json = pine_req.json()

## Navigating the Response

As with other responses to API calls, the taxonomy JSON at the uppermost level is a dictionary containing only an element with the 'data' label. Within this, the data is list with entries for each species returned by the call. Each species entry is a dictionary with atttributes for:

- The full taxonomy, with a seperate attribute for each taxonomic level
- The NEON taxonomy the data was obtained from (taxonTypeCode)
- The short taxon code used by NEON (taxonID, acceptedTaxonID)
- The author of the scientific name
- The common/vernbacular name, if any
- The reference text used (nameAccordingToID)

In [16]:
#Print data for one species in the result
sample = pine_json['data'][7]
for key in sample.keys():
    print("%28s:   %20s" % (key, sample[key]))

               taxonTypeCode:                  PLANT
                     taxonID:                   ABFI
             acceptedTaxonID:                   ABFI
          dwc:scientificName:   Abies firma Siebold & Zucc.
dwc:scientificNameAuthorship:        Siebold & Zucc.
               dwc:taxonRank:                species
          dwc:vernacularName:               Momi fir
       dwc:nameAccordingToID:   http://plants.usda.gov (accessed 8/25/2014)
                 dwc:kingdom:                Plantae
                  dwc:phylum:          Coniferophyta
                   dwc:class:              Pinopsida
                   dwc:order:                Pinales
                  dwc:family:               Pinaceae
                   dwc:genus:                  Abies
             gbif:subspecies:                   None
                gbif:variety:                   None


The "dwc" at the beginning of many atttribute names indicates that the terms used for each field are matched to those used by Darwin Core, an official standard maintained for biodiversity reference.

In [17]:
#Print vernacular and species names
for species in pine_json['data']:
    print(species['dwc:vernacularName'],'|',species['dwc:scientificName'])

silver fir | Abies alba Mill.
Pacific silver fir | Abies amabilis (Douglas ex Loudon) Douglas ex Forbes
balsam fir | Abies balsamea (L.) Mill.
balsam fir | Abies balsamea (L.) Mill. var. balsamea
balsam fir | Abies balsamea (L.) Mill. var. phanerolepis Fernald
bristlecone fir | Abies bracteata (D. Don) D. Don ex Poit.
white fir | Abies concolor (Gord. & Glend.) Lindl. ex Hildebr.
Momi fir | Abies firma Siebold & Zucc.
Fraser fir | Abies fraseri (Pursh) Poir.
grand fir | Abies grandis (Douglas ex D. Don) Lindl.
Nikko fir | Abies homolepis Siebold & Zucc.
fir | Abies sp.
fir | Abies spp.
subalpine fir | Abies lasiocarpa (Hook.) Nutt.
corkbark fir | Abies lasiocarpa (Hook.) Nutt. var. arizonica (Merriam) Lemmon
subalpine fir | Abies lasiocarpa (Hook.) Nutt. var. lasiocarpa
Sierra white fir | Abies lowiana (Gordon & Glend.) A. Murray bis
California red fir | Abies magnifica A. Murray bis
California red fir | Abies magnifica A. Murray bis var. magnifica
Shasta red fir | Abies magnifica A. M

## Using Taxon Type Code

Let's try another, using taxonTypeCode this time. We'll look through some of the NEON Fish Taxonomy, but try the verbose description.

In [18]:
#Set options
SERVER = 'http://data.neonscience.org/api/v0/'
TAXONCODE = 'FISH'
OFFSET = 0
LIMIT = 20
VERBOSE = 'true'

In [19]:
#Create 'options' portion of API call
OPTIONS = '?taxonTypeCode={taxoncode}&offset={offset}&limit={limit}&verbose={verbose}'.format(
    taxoncode = TAXONCODE,
    offset = OFFSET,
    limit = LIMIT,
    verbose = VERBOSE)
print(OPTIONS)

?taxonTypeCode=FISH&offset=0&limit=20&verbose=true


In [20]:
#Make request
fish_req = requests.get(SERVER+'taxonomy/'+OPTIONS)
fish_json = fish_req.json()

In [21]:
#Print data for one species in the result
sample = fish_json['data'][7]
for key in sample.keys():
    print("%28s:   %20s" % (key, sample[key]))

               taxonTypeCode:                   FISH
                     taxonID:                 ACHSPP
             acceptedTaxonID:                 ACHSPP
          dwc:scientificName:         Achiridae spp.
dwc:scientificNameAuthorship:                   None
               dwc:taxonRank:                 family
          dwc:vernacularName:                   None
       taxonProtocolCategory:                 target
       dwc:nameAccordingToID:   http://www.itis.gov/ITISWebService/services/ITISService/getFullRecordFromTSN?tsn=202070 (accessed 08/31/2017)
    dwc:nameAccordingToTitle:   The Integrated Taxonomic Information System on-line database http://www.itis.gov/ITISWebService/services/ITISService/getFullRecordFromTSN?tsn=202070, accessed 31 Aug 2017
                 dwc:kingdom:               Animalia
             gbif:subkingdom:                   None
           gbif:infrakingdom:                   None
          gbif:superdivision:                   None
               gbif

This is a more verbose entry, so there are more attributes, though many lack values. The 'gbif' attributes indicate terms matched to those used by the Global Biodiversity Forum.

In [11]:
#Print common and scientific name for each fish
for species in fish_json['data']:
    print(species['dwc:vernacularName'],'|', species['dwc:scientificName'])

None | Acanthogobius flavimanus
None | Acantharchus pomotis
None | Acantharchus sp.
None | Acanthogobius sp.
None | Acantharchus spp.
None | Acanthogobius spp.
None | Achiridae sp.
None | Achiridae spp.
None | Acipenser brevirostrum
None | Acipenser fulvescens
None | Acipenser medirostris
None | Acipenser oxyrinchus
Gulf Sturgeon | Acipenser oxyrinchus desotoi
Atlantic sturgeon | Acipenser oxyrinchus oxyrinchus
Gulf Sturgeon | Acipenser oxyrhynchus desotoi
None | Acipenser oxyrhynchus
None | Acipenser sp.
None | Acipenseridae sp.
None | Acipenseriformes sp.
None | Acipenser spp.
